In [21]:
import torch
import pickle
import config
import os
import numpy as np
import evaluation as E

In [36]:
# Device
device = config.device
# device = torch.device('cuda:0')
# device = 'cpu'

# Prepare data
## Datasets
datasets = os.listdir('./Datasets/datasets/')
datasets = [d for d in datasets if d.endswith('.p')]
datasets.sort()

## Load data
names   = []
num_in  = []
num_out = []
X_trains = []
y_trains = []
X_valids = []
y_valids = []
X_tests = []
y_tests = []

for dataset in datasets:
    datapath = os.path.join('./Datasets/datasets/' + dataset)
    with open(datapath, 'rb') as f:
        data = pickle.load(f)
        
    X_test     = data['X_test']
    y_test     = data['y_test']
    data_name  = data['name']

    N_class    = data['n_class']
    N_feature  = data['n_feature']
    N_train    = X_train.shape[0]
    N_valid    = X_valid.shape[0]
    N_test     = X_test.shape[0]
    
    print('Loading', data_name, N_feature, N_class, N_train, N_valid, N_test)
    
    names.append(data_name)
    num_in.append(N_feature)
    num_out.append(N_class)
    
    X_tests.append(X_test.to(device))
    y_tests.append(y_test.to(device))

print('Finish data loading.')

Loading acuteinflammation 6 2 184 61 25
Loading acutenephritis 6 2 184 61 25
Loading balancescale 4 3 184 61 126
Loading blood 4 2 184 61 150
Loading breastcancer 9 2 184 61 58
Loading breastcancerwisc 9 2 184 61 140
Loading breasttissue 9 6 184 61 22
Loading ecoli 7 8 184 61 68
Loading energyy1 8 3 184 61 154
Loading energyy2 8 3 184 61 154
Loading fertility 9 2 184 61 21
Loading glass 9 6 184 61 43
Loading habermansurvival 3 2 184 61 62
Loading hayesroth 3 3 184 61 32
Loading ilpdindianliver 9 2 184 61 117
Loading iris 4 3 184 61 31
Loading mammographic 5 2 184 61 193
Loading monks1 6 2 184 61 111
Loading monks2 6 2 184 61 120
Loading monks3 6 2 184 61 110
Loading pima 8 2 184 61 154
Loading pittsburgbridgesMATERIAL 7 3 184 61 22
Loading pittsburgbridgesRELL 7 3 184 61 21
Loading pittsburgbridgesSPAN 7 3 184 61 18
Loading pittsburgbridgesTORD 7 2 184 61 20
Loading pittsburgbridgesTYPE 7 6 184 61 22
Loading postoperative 8 3 184 61 19
Loading seeds 7 3 184 61 43
Loading teaching 5 3 1

In [37]:
model_file = os.listdir(f'./result/super pNN hyparam tuning/model/')
model_file.sort()
model_file

['spnn_False_0.001_0',
 'spnn_False_0.001_1',
 'spnn_False_0.001_2',
 'spnn_False_0.001_3',
 'spnn_False_0.001_4',
 'spnn_False_0.001_5',
 'spnn_False_0.001_6',
 'spnn_False_0.001_7',
 'spnn_False_0.001_8',
 'spnn_False_0.001_9',
 'spnn_False_0.01_0',
 'spnn_False_0.01_1',
 'spnn_False_0.01_2',
 'spnn_False_0.01_3',
 'spnn_False_0.01_4',
 'spnn_False_0.01_5',
 'spnn_False_0.01_6',
 'spnn_False_0.01_7',
 'spnn_False_0.01_8',
 'spnn_False_0.01_9',
 'spnn_False_0.1_0',
 'spnn_False_0.1_1',
 'spnn_False_0.1_2',
 'spnn_False_0.1_3',
 'spnn_False_0.1_4',
 'spnn_False_0.1_5',
 'spnn_False_0.1_6',
 'spnn_False_0.1_7',
 'spnn_False_0.1_8',
 'spnn_False_0.1_9',
 'spnn_False_0.2_0',
 'spnn_False_0.2_1',
 'spnn_False_0.2_2',
 'spnn_False_0.2_3',
 'spnn_False_0.2_4',
 'spnn_False_0.2_5',
 'spnn_False_0.2_6',
 'spnn_False_0.2_7',
 'spnn_False_0.2_8',
 'spnn_False_0.2_9',
 'spnn_False_0.5_0',
 'spnn_False_0.5_1',
 'spnn_False_0.5_2',
 'spnn_False_0.5_3',
 'spnn_False_0.5_4',
 'spnn_False_0.5_5',
 'sp

In [38]:
normalizations = [True, False]
lrs = [0.001, 0.01, 0.1, 0.2, 0.5, 1]
Results = torch.zeros([2,6,10])

In [39]:
for n in range(len(normalizations)):
    normalization = normalizations[n]
    acc_factor = np.loadtxt('./result/seperate pNN/acc_factor.txt').flatten().tolist()
    train_factor = np.loadtxt('./result/seperate pNN/train_factor.txt').flatten().tolist()
    valid_factor = np.loadtxt('./result/seperate pNN/valid_factor.txt').flatten().tolist()
    if not normalization:
        train_factor = [1 for i in range(len(datasets))]
        valid_factor = [1 for i in range(len(datasets))]
    for lr in range(len(lrs)):
        slr = lrs[lr]
        for seed in range(10):
            file_name = f'./result/super pNN hyparam tuning/model/spnn_{normalization}_{slr}_{seed}'
            spnn = torch.load(file_name)
            prediction_tests = spnn(X_tests) 
            tests_acc = E.ACC(prediction_tests, y_tests, acc_factor)
            
            Results[n, lr, seed] = tests_acc

In [43]:
Results_mean = torch.mean(Results, dim=2)
Results_mean

tensor([[0.9369, 1.0815, 1.0106, 0.8708, 0.7434, 0.6776],
        [0.9361, 1.0751, 1.0010, 0.8542, 0.6887, 0.6006]])

In [44]:
Results_std = torch.std(Results, dim=2)
Results_std

tensor([[0.0348, 0.0215, 0.0480, 0.0263, 0.0395, 0.1098],
        [0.0241, 0.0220, 0.0426, 0.0345, 0.1036, 0.1260]])

with normalization, lr=0.01

In [ ]:
loss_1_sep = 0.1
loss_2_sep = 2

In [ ]:
loss_1 = 1.5 / loss_1_sep
loss_2 = 2 / loss_2_sep